In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from math import log
from decimal import Decimal
import altair as alt
import kagglehub as kh

# test jo

In [13]:
# nba endpoints

## get miami heat team_id
from nba_api.stats.static import teams

## get players
from nba_api.stats.static import players

from nba_api.stats.endpoints import BoxScoreTraditionalV2

from nba_api.stats.endpoints import PlayerIndex

## Offensive Stat Predictions
idea- pick an offensive player and his most likely defensive matchup, to predict possible game scores

### Get all Heat players, check if they are there

In [15]:
all_players=PlayerIndex().get_data_frames()[0]

In [16]:
all_players

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
0,1630173,Achiuwa,Precious,precious-achiuwa,1610612752,knicks,0,New York,Knicks,NYK,...,2020.0,1.0,20.0,1.0,2020,2024,6.7,5.9,1.0,Season
1,203500,Adams,Steven,steven-adams,1610612745,rockets,0,Houston,Rockets,HOU,...,2013.0,1.0,12.0,1.0,2013,2024,3.6,5.3,1.2,Season
2,1628389,Adebayo,Bam,bam-adebayo,1610612748,heat,0,Miami,Heat,MIA,...,2017.0,1.0,14.0,1.0,2017,2024,17.7,10.0,4.4,Season
3,1630534,Agbaji,Ochai,ochai-agbaji,1610612761,raptors,0,Toronto,Raptors,TOR,...,2022.0,1.0,14.0,1.0,2022,2024,10.0,3.7,1.5,Season
4,1630583,Aldama,Santi,santi-aldama,1610612763,grizzlies,0,Memphis,Grizzlies,MEM,...,2021.0,1.0,30.0,1.0,2021,2024,12.7,6.7,2.8,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,1629027,Young,Trae,trae-young,1610612737,hawks,0,Atlanta,Hawks,ATL,...,2018.0,1.0,5.0,1.0,2018,2024,24.0,3.1,11.5,Season
563,1627826,Zubac,Ivica,ivica-zubac,1610612746,clippers,0,LA,Clippers,LAC,...,2016.0,2.0,32.0,1.0,2016,2024,16.2,12.5,2.5,Season
564,1641783,da Silva,Tristan,tristan-da-silva,1610612753,magic,0,Orlando,Magic,ORL,...,2024.0,1.0,18.0,1.0,2024,2024,7.6,3.5,1.6,Season
565,1628427,Čančar,Vlatko,vlatko-čančar,1610612743,nuggets,0,Denver,Nuggets,DEN,...,2017.0,2.0,49.0,1.0,2019,2024,1.5,1.7,0.0,Season


In [17]:
# get all players for miami heat
miami_heat_players = all_players[all_players['TEAM_CITY']=='Miami']
miami_heat_players

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
2,1628389,Adebayo,Bam,bam-adebayo,1610612748,heat,0,Miami,Heat,MIA,...,2017.0,1.0,14.0,1.0,2017,2024,17.7,10.0,4.4,Season
10,203937,Anderson,Kyle,kyle-anderson,1610612748,heat,0,Miami,Heat,MIA,...,2014.0,1.0,30.0,1.0,2014,2024,5.0,3.1,2.2,Season
65,202692,Burks,Alec,alec-burks,1610612748,heat,0,Miami,Heat,MIA,...,2011.0,1.0,12.0,1.0,2011,2024,6.4,2.7,1.1,Season
87,1630528,Christopher,Josh,josh-christopher,1610612748,heat,0,Miami,Heat,MIA,...,2021.0,1.0,24.0,1.0,2021,2024,0.8,0.4,0.5,Season
204,1629639,Herro,Tyler,tyler-herro,1610612748,heat,0,Miami,Heat,MIA,...,2019.0,1.0,13.0,1.0,2019,2024,23.9,5.4,5.7,Season
206,1629312,Highsmith,Haywood,haywood-highsmith,1610612748,heat,0,Miami,Heat,MIA,...,2018.0,NaN,NaN,1.0,2018,2024,6.5,3.2,1.4,Season
239,1631170,Jaquez Jr.,Jaime,jaime-jaquez-jr,1610612748,heat,0,Miami,Heat,MIA,...,2023.0,1.0,18.0,1.0,2023,2024,8.6,4.4,2.5,Season
251,1642352,Johnson,Keshad,keshad-johnson,1610612748,heat,0,Miami,Heat,MIA,...,NaN,NaN,NaN,1.0,2024,2024,1.6,1.7,0.3,Season
265,1631107,Jović,Nikola,nikola-jović,1610612748,heat,0,Miami,Heat,MIA,...,2022.0,1.0,27.0,1.0,2022,2024,10.7,3.9,2.8,Season
288,1641796,Larsson,Pelle,pelle-larsson,1610612748,heat,0,Miami,Heat,MIA,...,2024.0,2.0,44.0,1.0,2024,2024,4.0,1.5,1.0,Season


In [ ]:
 # keep only first 3 columns
miami_heat_players = miami_heat_players[['PERSON_ID','PLAYER_FIRST_NAME','PLAYER_LAST_NAME']]
miami_heat_players

,PERSON_ID,PLAYER_FIRST_NAME,PLAYER_LAST_NAME
2,1628389,Bam,Adebayo
10,203937,Kyle,Anderson
65,202692,Alec,Burks
87,1630528,Josh,Christopher
204,1629639,Tyler,Herro
206,1629312,Haywood,Highsmith
239,1631170,Jaime,Jaquez Jr.
251,1642352,Keshad,Johnson
265,1631107,Nikola,Jović
288,1641796,Pelle,Larsson


### Get all Heat Games

In [21]:

nba_teams = teams.get_teams()

miami = [team for team in nba_teams if team['abbreviation'] == 'MIA'][0]
miami_id = miami['id']

In [22]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=miami_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612748,MIA,Miami Heat,0022400948,2025-03-12,MIA vs. LAC,L,239,104,...,0.714,9,20,29,27,14,3,10,17,-15.0
1,22024,1610612748,MIA,Miami Heat,0022400931,2025-03-10,MIA vs. CHA,L,240,102,...,0.700,11,35,46,24,7,3,16,15,-3.0
2,22024,1610612748,MIA,Miami Heat,0022400916,2025-03-08,MIA vs. CHI,L,241,109,...,0.737,7,37,44,25,7,5,9,17,-5.0
3,22024,1610612748,MIA,Miami Heat,0022400907,2025-03-07,MIA vs. MIN,L,238,104,...,0.727,7,34,41,29,7,2,9,16,-2.0
4,22024,1610612748,MIA,Miami Heat,0022400893,2025-03-05,MIA @ CLE,L,239,107,...,0.880,9,33,42,29,7,2,13,19,-5.0


In [39]:
# Subset the games to when the last 4 digits of SEASON_ID were 2024
games_2024 = games[games.SEASON_ID.str[-4:] == '2024']
games_2024

games_2024_test = games_2024.head(5)
games_2024_test

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612748,MIA,Miami Heat,0022400948,2025-03-12,MIA vs. LAC,L,239,104,...,0.714,9,20,29,27,14,3,10,17,-15.0
1,22024,1610612748,MIA,Miami Heat,0022400931,2025-03-10,MIA vs. CHA,L,240,102,...,0.700,11,35,46,24,7,3,16,15,-3.0
2,22024,1610612748,MIA,Miami Heat,0022400916,2025-03-08,MIA vs. CHI,L,241,109,...,0.737,7,37,44,25,7,5,9,17,-5.0
3,22024,1610612748,MIA,Miami Heat,0022400907,2025-03-07,MIA vs. MIN,L,238,104,...,0.727,7,34,41,29,7,2,9,16,-2.0
4,22024,1610612748,MIA,Miami Heat,0022400893,2025-03-05,MIA @ CLE,L,239,107,...,0.880,9,33,42,29,7,2,13,19,-5.0


### importing common box score numbers

In [40]:
# get boxscores for all game ids in 2024
game_ids_2024_test = games_2024_test['GAME_ID'].values

# make a dictionary where game_id is the key and boxscore is value
boxscores = {}
for game_id in game_ids_2024_test:
    boxscores[game_id] = BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]



In [42]:
boxscores['0022400948']

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022400948,1610612746,LAC,LA,203992,Bogdan Bogdanović,Bogdan,F,,38.000000:34,...,0.0,10.0,10.0,4.0,0.0,0.0,5.0,1.0,30.0,19.0
1,0022400948,1610612746,LAC,LA,201587,Nicolas Batum,Nicolas,F,,32.000000:52,...,0.0,2.0,2.0,2.0,1.0,0.0,1.0,1.0,6.0,4.0
2,0022400948,1610612746,LAC,LA,1627826,Ivica Zubac,Ivica,C,,38.000000:00,...,6.0,8.0,14.0,1.0,0.0,0.0,2.0,2.0,26.0,5.0
3,0022400948,1610612746,LAC,LA,1627739,Kris Dunn,Kris,G,,23.000000:50,...,1.0,1.0,2.0,3.0,3.0,0.0,1.0,2.0,4.0,4.0
4,0022400948,1610612746,LAC,LA,201935,James Harden,James,G,,38.000000:31,...,3.0,5.0,8.0,11.0,2.0,0.0,3.0,1.0,24.0,15.0
5,0022400948,1610612746,LAC,LA,1627884,Derrick Jones Jr.,Derrick,,,23.000000:26,...,0.0,4.0,4.0,0.0,1.0,0.0,2.0,0.0,8.0,5.0
6,0022400948,1610612746,LAC,LA,1629599,Amir Coffey,Amir,,,25.000000:16,...,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,13.0,5.0
7,0022400948,1610612746,LAC,LA,1629234,Drew Eubanks,Drew,,,9.000000:26,...,0.0,2.0,2.0,1.0,0.0,0.0,1.0,2.0,6.0,8.0
8,0022400948,1610612746,LAC,LA,201988,Patty Mills,Patty,,,2.000000:17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
9,0022400948,1610612746,LAC,LA,1641738,Kobe Brown,Kobe,,,0.000000:34,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
